<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plotting-Code:" data-toc-modified-id="Plotting-Code:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plotting Code:</a></span><ul class="toc-item"><li><span><a href="#Flux-Power-Sweep" data-toc-modified-id="Flux-Power-Sweep-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Flux-Power Sweep</a></span><ul class="toc-item"><li><span><a href="#Trap-and-Release-Rate-with-fitting" data-toc-modified-id="Trap-and-Release-Rate-with-fitting-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Trap and Release Rate with fitting</a></span><ul class="toc-item"><li><span><a href="#Fitting" data-toc-modified-id="Fitting-1.1.1.1"><span class="toc-item-num">1.1.1.1&nbsp;&nbsp;</span>Fitting</a></span></li></ul></li></ul></li><li><span><a href="#Trap-Lifetime" data-toc-modified-id="Trap-Lifetime-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Trap Lifetime</a></span></li><li><span><a href="#Power-Sweep" data-toc-modified-id="Power-Sweep-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Power Sweep</a></span><ul class="toc-item"><li><span><a href="#Mean-Occupations" data-toc-modified-id="Mean-Occupations-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Mean Occupations</a></span></li><li><span><a href="#Probabilities-of-Mode" data-toc-modified-id="Probabilities-of-Mode-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Probabilities of Mode</a></span></li><li><span><a href="#Transition-Rates" data-toc-modified-id="Transition-Rates-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Transition Rates</a></span></li><li><span><a href="#Lifetimes" data-toc-modified-id="Lifetimes-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Lifetimes</a></span></li></ul></li><li><span><a href="#Lifetime-Statistics" data-toc-modified-id="Lifetime-Statistics-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Lifetime Statistics</a></span></li></ul></li></ul></div>

# Plotting Code:

## Flux-Power Sweep


In [ ]:
import numpy as np
import h5py
import fitTools.quasiparticleFunctions as qp
import matplotlib.pyplot as plt
import glob
plt.rcParams.update({'axes.labelpad':0.2,
                     'axes.linewidth':1.0,
                     'figure.dpi':300.0,
                     'figure.figsize':[3.325,3.325],
                     'legend.frameon':True,
                     'legend.handlelength':1.0,
                     'xtick.major.pad':2,
                     'xtick.minor.pad':2,
                     'xtick.major.width':1.0,
                     'ytick.major.pad':2,
                     'ytick.minor.pad':2,
                     'ytick.major.width':1.0,
                     'axes.ymargin':0.01,
                     'axes.xmargin':0.01})
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.size'] = 24
figurepath = os.path.join(SPATH,'autoPhi\Figures')

if not os.path.exists(figurepath):
    os.makedirs(figurepath)

import os
from matplotlib.colors import LogNorm
from matplotlib.patches import Ellipse
from time import perf_counter, sleep
from hmmlearn import hmm
import pickle

###  Trap and Release Rate with fitting


In [ ]:
lPhi = np.round(np.arange(0.5,0.3,-0.001),decimals=3)
lPowers = np.round(np.arange(-156,-126.9),decimals=0)
lRate = np.empty((len(lPhi),len(lPowers)))

with h5py.File(savefile,'r') as f:
    for phi in lPhi:
        g = f[f'PHI{phi*1000:3.0f}']
        fullrates = np.empty(lPowers.shape)
        fullrates.fill(np.nan)
        powers = np.array([g[k].attrs.get('LOpower') for k in g])
        psort = np.argsort(powers)
        mask = np.intersect1d(lPowers,powers,return_indices=True)[1]
        rates = np.array([g[k]['transitionRatesMHz'][0,1] for k in g])
        fullrates[mask] = rates[psort]
        lRate[lPhi == np.round(g.attrs['phi'],decimals=3),:] = fullrates
#         plt.plot(lPowers,fullmeans)
#         plt.title(f'phi = {phi}')
#         plt.show()
#         plt.close()
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,lPhi)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\phi$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('Trap rate [MHz]')
plt.show()

In [ ]:
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,trapdepth)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\Delta - E_A [GHz]$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('Trap rate [MHz]')
plt.show()

In [ ]:
import matplotlib.ticker as mticker
def log_tick_formatter(val, pos=None):
    return f"$10^{{{np.round(val,decimals=1)}}}$"  # remove int() if you don't use MaxNLocator
    # return f"{10**val:.2e}"      # e-Notation

lPhi = np.round(np.arange(0.5,0.3,-0.001),decimals=3)
lPowers = np.round(np.arange(-156,-126.9),decimals=0)
lRate = np.empty((len(lPhi),len(lPowers)))

with h5py.File(savefile,'r') as f:
    for phi in lPhi:
        g = f[f'PHI{phi*1000:3.0f}']
        fullrates = np.empty(lPowers.shape)
        fullrates.fill(np.nan)
        powers = np.array([g[k].attrs.get('LOpower') for k in g])
        psort = np.argsort(powers)
        mask = np.intersect1d(lPowers,powers,return_indices=True)[1]
        rates = np.array([g[k]['transitionRatesMHz'][1,0] for k in g])
        fullrates[mask] = rates[psort]
        lRate[lPhi == np.round(g.attrs['phi'],decimals=3),:] = fullrates
#         plt.plot(lPowers,fullmeans)
#         plt.title(f'phi = {phi}')
#         plt.show()
#         plt.close()
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,lPhi)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\phi$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('Release rate [MHz]')
plt.show()

In [ ]:
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,trapdepth)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\Delta - E_A [GHz]$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('Release rate [MHz]')
plt.show()

#### Fitting

In [ ]:
from scipy.optimize import curve_fit
def curves(p,a,b,c):
    return a + b*p + c*np.sqrt(p)

In [ ]:
lPhi = np.round(np.arange(0.5,0.3,-0.001),decimals=3)
lPowers = np.round(np.arange(-156,-126.9),decimals=0)
lRate = np.empty((len(lPhi),len(lPowers)))
lCurves = np.empty((len(lPhi),len(lPowers)))
lPars = np.empty((len(lPhi),3))
lError = np.empty_like(lPars)
lMask = np.ones(len(lPhi),dtype=bool)
with h5py.File(savefile,'r') as f:
    for phi in lPhi:
        g = f[f'PHI{phi*1000:3.0f}']
        fullrates = np.empty(lPowers.shape)
        fullrates.fill(np.nan)
        powers = np.array([g[k].attrs.get('LOpower') for k in g])
        psort = np.argsort(powers)
        mask = np.intersect1d(lPowers,powers,return_indices=True)[1]
        rates = np.array([g[k]['transitionRatesMHz'][1,0] for k in g])
        guess = [0.025,1e12,1e5]
        pars,cov = curve_fit(curves,10**(powers[psort]/10),rates[psort],p0=guess,bounds=(1e-3,np.inf),maxfev=5000)
        lPars[lPhi == np.round(g.attrs['phi'],decimals=3),:] = pars
        lError[lPhi == np.round(g.attrs['phi'],decimals=3),:] = np.sqrt(np.diag(cov))/pars
#         if any(np.sqrt(np.diag(cov))/pars > 1):
#             lMask[lPhi == np.round(g.attrs['phi'],decimals=3)] = False
#         print(pars)
#         print(f'{np.sqrt(np.diag(cov))/pars} \n')
        fullrates[mask] = rates[psort]
        lCurves[lPhi == np.round(g.attrs['phi'],decimals=3),:] = curves(10**(lPowers/10),*pars)
#         fig,ax = plt.subplots(figsize=[9,6])
#         ax.plot(powers[psort],rates[psort],label='data')
#         ax.plot(lPowers,curves((10**(lPowers/10)),*pars),label=f'{pars}')
#         ax.set_yscale('log')
#         plt.title(f"{np.round(g.attrs['phi'],decimals=3)}")
#         plt.legend()
#         plt.show()
        lRate[lPhi == np.round(g.attrs['phi'],decimals=3),:] = fullrates
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)
XX,YY = np.meshgrid(lPowers,lPhi)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
ax.plot_wireframe(XX, YY, np.log10(lCurves), rstride=1, cstride=1,alpha=0.4)
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\phi$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('release rate [MHz]')
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=[9,6])
ax.plot(lPhi[lMask],lPars[:,0][lMask],label='intercept.')
ax.plot(lPhi[lMask],lPars[:,1][lMask],label='lin coeff.')
ax.plot(lPhi[lMask],lPars[:,2][lMask],label='sqrt coeff.')
ax.set_yscale('log')
plt.legend()
plt.show()

## Trap Lifetime

In [ ]:
lPhi = np.round(np.arange(0.5,0.3,-0.001),decimals=3)
lPowers = np.round(np.arange(-156,-126.9),decimals=0)
lRate = np.empty((len(lPhi),len(lPowers)))

with h5py.File(savefile,'r') as f:
    for phi in lPhi:
        g = f[f'PHI{phi*1000:3.0f}']
        fullrates = np.empty(lPowers.shape)
        fullrates.fill(np.nan)
        powers = np.array([g[k].attrs.get('LOpower') for k in g])
        psort = np.argsort(powers)
        mask = np.intersect1d(lPowers,powers,return_indices=True)[1]
        rates = np.array([1/g[k]['transitionRatesMHz'][1,1] for k in g])
        fullrates[mask] = rates[psort]
        lRate[lPhi == np.round(g.attrs['phi'],decimals=3),:] = fullrates
#         plt.plot(lPowers,fullmeans)
#         plt.title(f'phi = {phi}')
#         plt.show()
#         plt.close()
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,lPhi)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.set_ylabel('$\phi$')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('trap lifetime [$\mu$s]')
plt.show()

In [ ]:
Ea_D = np.sqrt(1-np.sin(np.pi*lPhi/2)**2)
trapdepth = (1 - Ea_D)*50
XX,YY = np.meshgrid(lPowers,trapdepth)

fig,ax = plt.subplots(figsize=[9,6],subplot_kw={'projection':'3d'})
surf = ax.plot_surface(XX,YY,np.log10(lRate),cmap = cm.coolwarm)
ax.zaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.set_ylabel('$\Delta - E_A$ [GHz]')
ax.set_xlabel('Readout power [dBm]')
ax.set_zlabel('trap lifetime [$\mu$s]')
plt.show()

## Power Sweep

In [ ]:
ints = 0
ks = f'{ints}QP'.replace('.','p')
dsr = f'{sr}MHz'.replace('.','p')

savefile = r"E:\NBR07_powerSweeps\ProcessedData\cooldown7_dataset1.hdf5"
with h5py.File(savefile,'a') as f:
    for key in metainfo:
        f.attrs.create(key,metainfo[key])

figpath = r"E:\NBR07_powerSweeps\Figures\cooldown7_dataset1\{}QP\\".format(ks)
import os
if not os.path.exists(figpath):
    os.makedirs(figpath)

### Mean Occupations

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        Qmean = f[key].attrs.get('mean')
        plt.scatter(LOp,Qmean,color='red')
plt.xlabel('LO power [dBm]')
plt.ylabel('Mean occupation')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.savefig(figpath + 'meanOccupation.png')

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        Qmean = f[key].attrs.get('mean')
        plt.scatter(LOp,Qmean,color='red')
plt.xlabel('LO power [dBm]')
plt.ylabel('Mean occupation')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.yscale('log')
plt.savefig(figpath + 'meanOccupation_logscale.png')

### Probabilities of Mode

In [ ]:
from matplotlib.lines import Line2D

with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        P0 = f[key].attrs.get('P0')
        P1 = f[key].attrs.get('P1')
        P2 = f[key].attrs.get('P2')
        plt.scatter(LOp,P0,color='purple')
        plt.scatter(LOp,P1,color='orange')
        plt.scatter(LOp,P2,color='green')
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='P0'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='P1'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='P2')]
plt.xlabel('LO power [dBm]')
plt.ylabel('Probability of mode')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.legend(handles=legend_elements)
plt.savefig(figpath + 'probabilities.png')

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        P0 = f[key].attrs.get('P0')
        P1 = f[key].attrs.get('P1')
        P2 = f[key].attrs.get('P2')
        plt.scatter(LOp,P0,color='purple')
        plt.scatter(LOp,P1,color='orange')
        plt.scatter(LOp,P2,color='green')
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='P0'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='P1'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='P2')]
plt.xlabel('LO power [dBm]')
plt.ylabel('Probability of mode')
plt.yscale('log')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.legend(handles=legend_elements)
plt.savefig(figpath + 'probabilities_logscale.png')

### Transition Rates

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        rates = f[key]['transitionRatesMHz'][:]
        plt.scatter(LOp,rates[0,1],color='purple')
        plt.scatter(LOp,rates[1,2],color='orange')
        plt.scatter(LOp,rates[0,2],color='green')
        plt.scatter(LOp,rates[2,1],color='cyan')
        plt.scatter(LOp,rates[1,0],color='red')
        plt.scatter(LOp,rates[2,0],color='magenta')
        
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='$\\Gamma_{01}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='$\\Gamma_{12}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='$\\Gamma_{02}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='cyan', label='$\\Gamma_{21}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='red', label='$\\Gamma_{10}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='magenta', label='$\\Gamma_{20}$')]
plt.xlabel('LO power [dBm]')
plt.ylabel('Transition rate [MHz]')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.legend(handles=legend_elements)
plt.ylim(0,0.2)
plt.savefig(figpath + 'transitionrates.png')
plt.show()
plt.close()

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        rates = f[key]['transitionRatesMHz'][:]
        plt.scatter(LOp,rates[0,1],color='purple')
        plt.scatter(LOp,rates[1,2],color='orange')
        plt.scatter(LOp,rates[0,2],color='green')
        plt.scatter(LOp,rates[2,1],color='cyan')
        plt.scatter(LOp,rates[1,0],color='red')
        plt.scatter(LOp,rates[2,0],color='magenta')
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='$\\Gamma_{01}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='$\\Gamma_{12}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='$\\Gamma_{02}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='cyan', label='$\\Gamma_{21}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='red', label='$\\Gamma_{10}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='magenta', label='$\\Gamma_{20}$')]
plt.xlabel('LO power [dBm]')
plt.ylabel('Transition rate [MHz]')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.yscale('log')
plt.legend(handles=legend_elements)
plt.savefig(figpath + 'transitionrates_logscale.png')
plt.show()
plt.close()

### Lifetimes

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        rates = f[key]['transitionRatesMHz'][:]
        plt.scatter(LOp,1/rates[0,0],color='purple')
        plt.scatter(LOp,1/rates[1,1],color='orange')
        plt.scatter(LOp,1/rates[2,2],color='green')
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='$\\tau_{0}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='$\\tau_{1}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='$\\tau_{2}$')]
plt.xlabel('LO power [dBm]')
plt.ylabel('lifetime [$\mu#s]')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.legend(handles=legend_elements)
plt.savefig(figpath + 'lifetimes.png')
plt.show()
plt.close()

In [ ]:
with h5py.File(savefile,'r') as fb:
    f = fb[ks]
    for key in f:
        LOp = f[key].attrs.get('LOpower')
        rates = f[key]['transitionRatesMHz'][:]
        plt.scatter(LOp,1/rates[0,0],color='purple')
        plt.scatter(LOp,1/rates[1,1],color='orange')
        plt.scatter(LOp,1/rates[2,2],color='green')
legend_elements = [Line2D([0], [0], marker='o', color='w',markerfacecolor='purple', label='$\\tau_{0}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='orange', label='$\\tau_{1}$'),
                  Line2D([0], [0], marker='o', color='w',markerfacecolor='green', label='$\\tau_{2}$')]
plt.xlabel('LO power [dBm]')
plt.ylabel('lifetime [$\mu#s]')
plt.title(f'Cooldown 7 | 22 mK | {ints} QP LO')
plt.yscale('log')
plt.legend(handles=legend_elements)
plt.savefig(figpath + 'lifetimes_logscale.png')
plt.show()
plt.close()

## Lifetime Statistics